In [1]:
import os
import sys
import re
import pandas as pd
import json
import math 

from collections import Counter
import matplotlib.pyplot as plt

In [2]:
country = 'usa'
news = open(f"seed_channel_list/{country}_news_channel_list.json", 'r', encoding="utf-8")
sci = open(f"seed_channel_list/science_channel_list.json", 'r', encoding="utf-8")
df = pd.read_json(sci)
channels, urls = [], []
for channel, url in df["channels"]:
    channels.append(channel.replace(" ",""))
    urls.append(url)
    
print(len(channels))
len(set(urls))


138


138

In [3]:
pattern = r"(?<=output_).[^_]*(?=_)"
dir_name = f"outputs_{country}_news"
dir_name = "outputs_science"

channel_list = set()
files = [f"{dir_name}/{f}" for f in os.listdir(f"{dir_name}/") if f.endswith(".csv")]

for f in files:
    channel_name = re.search(pattern, f)[0]
    channel_list.add(channel_name)


In [4]:
all_df_list = []
for channel in channel_list:
    channel_files = [f for f in files if f.startswith(f"{dir_name}/output_{channel}")]
    channel_df_list = []

    for f in channel_files:
        df_temp = pd.read_csv(f, encoding= 'unicode_escape')
        channel_df_list.append(df_temp)

    df = pd.concat(channel_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
    all_df_list.append(df)
    
df = pd.DataFrame()
df = pd.concat(all_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
df

,video_id,channel_name,video_title
0,oAVjF_7ensg,DeepAstronomy,The Hubble Ultra Deep Field in 3D
1,fgg2tpUVbXQ,DeepAstronomy,Hubble Deep Field: The Most Imp. Image Ever T...
2,ah6KrqABzmk,DeepAstronomy,Seeing Beyond - The James Webb Space Telescope...
3,zqb1lSdqRZY,DeepAstronomy,How Do We Know the Universe is Flat?
4,M6_HooIJLNc,DeepAstronomy,The Importance of Gliese 581g
...,...,...,...
27692,Q1XFi9r3dIE,DeepLook,These Mites Rain Down To Save Your Strawberrie...
27693,UfMCZYkG5Xo,DeepLook,This Bee Builds Sandcastles at the Beach | Dee...
27694,b7LINcDwl3o,DeepLook,Why Reindeer and Their Cousins are Total Boneh...
27695,jUvJ5ANH86I,DeepLook,From Drifter to Dynamo: The Story of Plankton ...


In [5]:
Counter(df["channel_name"])

Counter({'GoogleforDevelopers': 2000,
         'SciShow': 1609,
         'InsiderTech': 1609,
         'LinusTechTips': 1034,
         'CrazyRussianHacker': 1021,
         'NASA': 911,
         'VideoFromSpace': 878,
         'NatGeoWILD': 859,
         'RCMEDIAWORLD': 723,
         'ReedTimmer': 576,
         'NASASpaceflight': 525,
         'HardwareCanucks': 523,
         'ScottManley': 497,
         'NOVAPBSOfficial': 478,
         'CaliforniaAcademyofSciences': 462,
         'YaleCourses': 449,
         'SciShowSpace': 447,
         'JonathanMorrison': 432,
         'CrashCourse': 395,
         'LockPickingLawyer': 394,
         'TKOR': 371,
         'Dave2D': 357,
         'MindYourDecisions': 322,
         'sentdex': 313,
         'freeCodeCamporg': 295,
         'Discovery': 287,
         'Techquickie': 279,
         'Bitwit': 270,
         'Seeker': 250,
         'WorldScienceFestival': 250,
         'NASAGoddard': 250,
         'NationalGeographic': 250,
         'TedEd': 250

In [40]:
# mapping of number of youtube users per country (DataReportal 2023)
pop_mapping = {
    'irl': 4.0 * 10**6,
    'usa': 246 * 10**6,
    'can': 33.1 * 10**6,
    'aus': 21.3 * 10**6
}

TARGET_VIDEO_COUNT = 1000
VIEW_COUNT_CUT_OFF = 0.001 * pop_mapping[country]

dff = df[df['view_count'] >= VIEW_COUNT_CUT_OFF].reset_index(drop=True)

num_round = round(TARGET_VIDEO_COUNT/len(channel_list))
video_list = pd.DataFrame()

while len(video_list) < TARGET_VIDEO_COUNT:
    for channel in channel_list:
        dft = dff[dff['channel_name'] == channel]
        dft = dft.sample(n=min(len(dft), num_round))
        
        dff.drop(dff[dff['video_id'].isin(dft['video_id'])].index, inplace=True)
            
        video_list = pd.concat([dft, video_list]).reset_index(drop=True)
        video_list.drop_duplicates(subset='video_id', keep="last", inplace=True)

print(len(video_list))
display(Counter(video_list["channel_name"]))
video_list = video_list.sample(frac=1, random_state=2).reset_index(drop=True)
display(video_list)

1007


Counter({'kcalnews': 228,
         'CBSNews': 228,
         'CourtTV': 96,
         'ClickOnDetroit': 78,
         'WUSA9news': 59,
         'fox26houston': 56,
         'Denver7': 53,
         'FOX35Orlando': 38,
         'Fox10Phoenix': 38,
         '8NewsNowVegas': 36,
         'NBC10Philadelphia': 23,
         'harvardbusinessreview': 11,
         'WBIR10': 9,
         'KOMONews': 9,
         'KGUN9': 9,
         'AustinAmerican-Statesman': 5,
         'KiviTVNews': 4,
         'KOB': 3,
         'wpdeabc15': 3,
         'HawaiiNewsNow': 3,
         'WNEMTV5': 2,
         'TimesandDemocrat': 2,
         'YorkDailyRecord': 1,
         'KCRG': 1,
         'KTTC': 1,
         'USCourts': 1,
         'ActionNewsNow': 1,
         'TheDailyRecord': 1,
         'WHIO7': 1,
         'VerifyThis': 1,
         'IowasNewsNow': 1,
         'NorfolkDailyNews': 1,
         'CasperStarTribune': 1,
         'TheMarshallProject': 1,
         'USGAO': 1,
         'MotleyFool': 1})

,video_id,channel_name,video_title,view_count
0,wMEeCOUlyWM,CBSNews,Why Republicans continue to back Trump amid im...,681000
1,0Xv4Mqj4vio,Denver7,Man gored by elk while taking video,1000000
2,4pBNG1JMLpo,CBSNews,S.F. police release video of deadly shooting a...,691000
3,wPgxIeUGWWk,CBSNews,Hina Rabbani Khar: What happens in Afghanistan...,1000000
4,5DIad2tuVtg,kcalnews,Illegal Fireworks Boom Throughout Los Angeles ...,406000
...,...,...,...,...
1002,hPG6JjuxjWE,NBC10Philadelphia,"Video Shows Dirt Bike Rider Attacking Driver, ...",445000
1003,1m4zdcOBgI0,CourtTV,Jury Sees Maya Kowalski in Therapy,273000
1004,yB4lHAEsSyo,CBSNews,The incredible Albert Pujols,692000
1005,guGxpw4kMm0,WUSA9news,"YouTuber shot in mall ""traumatized' woman week...",274000


In [6]:
output_file_name = f"video_list/{country}_news_Apr052024.csv"
output_file_name = f"video_list/sci_raw.csv"
df.to_csv(output_file_name, index=False, encoding="utf-8")